In [1]:
import folium

In [2]:
%run '../scripts/ETL.py' '../scripts/paths.json'

/Users/Kasturi/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


22/10/07 13:09:58 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/10/07 13:09:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 13:09:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/07 13:09:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
final_join3.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,suburb,postcodes,long,lat,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021,fraud_probability_consumer,fraud_probability_merchant
null,null,null,null,null,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,32234779638,39.304036732369084,02e36af7-8c70-436...,2021-11-30,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01
Lorem Ipsum Sodal...,43186523025,florists supplies...,4.47,b,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,43186523025,56.35375319523918,a1d3526c-f212-41f...,2022-06-06,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01
Pede Blandit LLP,28517327395,"books, periodical...",0.47,e,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,28517327395,276.3597685809581,b90e183b-25d4-4a6...,2022-06-14,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01
Nullam Suscipit LLC,86388904574,"computers, comput...",4.96,b,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,86388904574,121.9799348513232,4c55adbf-7114-4bd...,2022-05-24,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01
Phasellus At Limited,76767266140,"furniture, home f...",4.65,b,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,76767266140,59.46778996058449,4e66a3c6-9691-44a...,2022-03-05,18983,36103,REVESBY,2212,151.014547,-33.947286,119011360,119011360,Revesby,576485403,8603,9061,17664,1,New South Wales,17719,17710,0.01,0.01


In [ ]:
final_join3.select("postcodes").distinct().count()

2343

In [ ]:
num_transactions_by_postcode = final_join3.groupBy(['postcodes', 'suburb', 'long', 'lat']).count()

In [ ]:
# conver transactions location to pandas df
num_transactions_by_postcode_pdf = num_transactions_by_postcode.toPandas()

aus_coords = [-25.2744, 133.7751]
m = folium.Map(aus_coords, tiles='OpenStreetMap', zoom_start=4.5)

for index, row in num_transactions_by_postcode_pdf.iterrows():
    if row['count'] >= 10000:
        marker_color = 'darkred'
        fill_color = 'darkred'
    elif row['count'] >= 5000:
        marker_color = 'red'
        fill_color = 'red'
    elif row['count'] >= 500:
        marker_color = 'darkorange'
        fill_color = 'darkorange'
    elif row['count'] >= 100:
        marker_color = 'orange'
        fill_color = 'orange'
    elif row['count'] <= 50 :
        marker_color = 'yellow'
        fill_color = 'yellow'
    else:
        marker_color='darkpurple'
        fill_color = 'darkpurple'
        
    folium.Circle(
          location=[row['lat'], row['long']],
          popup= 'Number of transactions: ' +str(row['count']),
          tooltip=row['suburb'],
          radius=row['count'],
          color=marker_color,
          fill=True,
          fill_color=fill_color,
       ).add_to(m)
m.save('../plots/bubble_plot_num_transactions_by_location.html')

In [ ]:
%run '../scripts/fraud_feature.py' '../scripts/paths.json'

In [ ]:

fraud_by_postcode = pd.read_parquet('../data/tables/full_dataset_with_fraud_rates.parquet')
fraud_by_postcode.head(5)

,merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,sus_merchant_abn,average fraud rate per merchant
0,None,NaN,None,NaN,None,Jose Frye,394 Matthews View Suite 660,NSW,2021,Male,8.078817e+10,86.718630,1c60001e-77cd-4755-9a89-265b4f197dc5,2021-04-21,5156.0,19115.0,NaN,NaN
1,None,NaN,None,NaN,None,Mrs. Valerie Powell,23459 Nicole Green Suite 420,VIC,3194,Female,8.078817e+10,1.572543,459f4f62-3953-4318-8de1-4b0bbc673cf1,2021-07-07,8323.0,69735.0,NaN,NaN
2,None,NaN,None,NaN,None,Maria Miller,3893 Martin Knoll Suite 887,NSW,2038,Undisclosed,8.078817e+10,48.925310,25d0ea41-da85-4930-aa25-7abedad2da8a,2021-03-25,3635.0,72910.0,NaN,NaN
3,None,NaN,None,NaN,None,Alexander Skinner,35694 Kimberly Coves Apt. 057,VIC,3053,Male,8.078817e+10,168.737463,1a5ffb5a-2d40-4127-8377-7e82409712f1,2021-12-30,9248.0,147322.0,NaN,NaN
4,None,NaN,None,NaN,None,Alexander Skinner,35694 Kimberly Coves Apt. 057,VIC,3053,Male,8.078817e+10,127.266580,8bddc10d-14a4-4c8f-a904-ed03b76008d6,2021-09-21,9248.0,147322.0,NaN,NaN
